### RAG Evaluation


In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os

This cell runs a sample query against the RAG system and prints both the generated answer and the retrieved document chunks.  
Use this output to manually evaluate:

- **Groundedness:** Is the answer clearly supported by the retrieved documents?
- **Relevance:** Does the answer directly address the user's question using the provided context?

We can use these criteria to assess the quality of your RAG pipeline.

In [3]:
load_dotenv()

CHROMA_PATH = "chroma_db"

# Reuse embeddings and load persisted vectorstore
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory=CHROMA_PATH,
)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 7}),
    memory=memory,
    verbose=True,
    max_tokens_limit=4096,
    return_source_documents=True
)

sample_question = "অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?"  # Example Bangla query
sample_history = []

result = qa_chain.invoke(
    {"question": sample_question, "chat_history": sample_history}
)

print("=== Query ===")
print(sample_question)
print("\n=== Answer ===")
print(result["answer"])
print("\n=== Retrieved Document Chunks ===")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n--- Chunk {i+1} ---\n{doc.page_content}\n")





> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
সমাধান:

ক. শম্ভুনাথ সেকরার হাতে একজোড়া এয়ারিং পরখ করতে দিয়েছিলেন।
খ. 'বাংলাদেশের মধ্যে আমিই একমাত্র পুরুষ যাহাকে কন্যার বাপ বিবাহ আসর হইতে নিজে ফিরাইয়া দিয়াছে'
বলতে অনুপমের আক্ষেপ ও অসহায়ত্বকে বোঝানো হয়েছে।
বাংলাদেশে বিয়েতে প্রায়শই দেখা যায় যে, প্রতিশ্রুত ও প্রদত্ত যৌতুকের অসংগতির কারণে বরের বাবা বিয়েতে
অসম্মতি জানায়। কিন্তু অনুপমের ক্ষেত্রে এর বিপরীত ঘটনা ঘটেছে। তার মামার যৌতুক গ্রহণের প্রবণতা,
লোভ এবং হীন মানসিকতার পরিচয় পেয়ে শম্ভুনাথ সেন মেয়ের আশীর্বাদের এয়ারিং ফিরিয়ে দেন এবং বিয়ে
ভেঙে দেন। এতে অনুপমের মনে হয়েছে শম্ভুনাথ বাবু যেন বর অনুপমকেই বিয়ের আসর থেকে ফিরিয়ে
দিয়েছেন যা বাংলাদেশে বিরল ঘটনা।

ঘ. অনুপমের মামা ও হারুন মিয়ার মতো মানুষের কারণে আজও কল্যাণী ও লাবনিরা অপমানের শি

#### Cosine Similarity Evaluation

This cell calculates the cosine similarity between the query embedding and each retrieved document chunk embedding.  
Cosine similarity provides a quantitative measure of how semantically close each chunk is to the user's question.

- **High similarity score** (close to 1) means the chunk is highly relevant to the query.
- **Low similarity score** means the chunk is less relevant.

Use these scores to assess the effectiveness of your retrieval system and to support your manual evaluation of



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Embed the query
query_embedding = embeddings.embed_query(sample_question)

# Get embeddings for retrieved chunks
chunk_embeddings = [embeddings.embed_documents([doc.page_content])[0] for doc in result["source_documents"]]

# Compute cosine similarity scores
scores = [cosine_similarity([query_embedding], [chunk_emb])[0][0] for chunk_emb in chunk_embeddings]

for i, score in enumerate(scores):
    print(f"Chunk {i+1} Cosine Similarity: {score:.4f}")

Chunk 1 Cosine Similarity: 0.5059
Chunk 2 Cosine Similarity: 0.4900
Chunk 3 Cosine Similarity: 0.4733
Chunk 4 Cosine Similarity: 0.4693
Chunk 5 Cosine Similarity: 0.4654
Chunk 6 Cosine Similarity: 0.4650
Chunk 7 Cosine Similarity: 0.4491


  Using cached scikit_learn-1.7.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.7 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 415.4 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
